In [1]:
import lxml.etree as ET
from tqdm import tqdm
import numpy as np
from gensim.models import KeyedVectors
import pandas as pd
from gensim.models import Word2Vec

In [2]:
drugbank_path = "G:\\CEGNN\\materials\\drugbank_all_full_database\\drugbank.xml"
drugbank = ET.parse(drugbank_path)
root = drugbank.getroot()
ns = {'db': 'http://www.drugbank.ca'}

In [3]:
drugs = root.xpath("db:drug[db:groups/db:group='approved']", namespaces=ns)

In [4]:
drug = drugs[0]

In [5]:
drugName = drug.find("db:name", ns).text
drugbank_id = drug.find("db:drugbank-id[@primary='true']", ns).text
drugDescription = drug.find("db:description", ns).text
cas_number = drug.find("db:cas-number", ns).text
unii = drug.find("db:unii", ns).text

if drugDescription is not None:
    drugDescription = drugDescription.replace("\n", "")
    drugDescription = drugDescription.replace("\r", "")
state = drug.find("db:state", ns)
if state is not None:
    state = state.text
else:
    state = "None"

drug_interactions = drug.xpath("db:drug-interactions/db:drug-interaction", namespaces=ns)
interactions_count = len(drug_interactions)

group = drug.xpath("db:groups/db:group", namespaces=ns)
group_name = ""
for g in group:
    group_name = group_name + "#" + g.text

clsf = drug.find("db:classification", ns)
direct_parent = clsf.find("db:direct-parent", ns)
kingdom = clsf.find("db:kingdom", ns)
superclass = clsf.find("db:superclass", ns)
cclass = clsf.find("db:class", ns)
subclass = clsf.find("db:subclass", ns)

uniprot_ids = drug.xpath("db:pathways/db:pathway/db:enzymes/db:uniprot-id", namespaces=ns)
uniprots = []
for uniprot_id in uniprot_ids:
    uniprots.append([drugbank_id, uniprot_id.text])

In [39]:
clsf = drug.find("db:classification", ns)

In [40]:
direct_parent = clsf.find("db:direct-parent", ns)

In [41]:
direct_parent.text

'Peptides'

In [ ]:
target = drug.xpath("db:targets/db:target", namespaces=ns)
if len(target) > 0:
    t = target[0]
    action = t.find("db:actions/db:action", ns)
    if action is not None:
        action = action.text
    else:
        action = "None"
else:
        action = "None"

In [13]:
from gensim.models import KeyedVectors
import pandas as pd
import os
import numpy as np
import gensim as gensim

In [18]:
def load_embeddings(nodes_list, embedding_file, dim=200):
    """
    针对gensim Word2vec模型保存的embedding文件进行读取
    """
    embeddings_model = gensim.models.KeyedVectors.load_word2vec_format(embedding_file, binary=False,
                                                                       unicode_errors='ignore')
    embedding_weights = []
    found = 0
    notfound = 0
    for node in nodes_list:
        node = int(node[2:])
        if str(node) in embeddings_model.vocab:
            embedding_weights.append(embeddings_model.word_vec(str(node)))
            found += 1
            # print(node)
        else:
            embedding_weights.append(np.random.uniform(-0.25, 0.25, dim).astype(np.float32))
            notfound += 1
    print("found_cnt size is :" + str(found))
    print("not found_cnt size is :" + str(notfound))
    return embedding_weights

In [9]:
dataset_path = "G:\\CEGNN\\materials\\drugbank"
dataset = "drugbank_action_labels.csv"
ids_labels = pd.read_csv(os.path.join(dataset_path, dataset), delimiter=",", dtype=np.dtype(str),
                             names=["ids", "labels"])

In [19]:
emb_path = "G:\\CEGNN\\materials\\emb\\drugbank"
inv_path = "G:\\CEGNN\\materials\\drugbank\\drugbank_index2id_edges.csv"
embedding = "drugbank_node2vec_200.emb"
nodes = ids_labels["ids"].tolist()
labels = ids_labels["labels"].tolist()
features = load_embeddings(nodes, os.path.join(emb_path, embedding), 200)

found_cnt size is :2408
not found_cnt size is :1580
